Pass Fail Predictor by Hiral

In [25]:
# ==============================
# PHASE 1: DATASET LOAD & VALIDATION
# ==============================

import pandas as pd

# ------------------------------
# Project Configuration
# ------------------------------

subjects = [
    "maths",
    "science",
    "english",
    "hindi",
    "social_science",
    "computer_science"
]

MARK_LIMITS = {
    "midsem": 25,
    "quiz": 10,
    "internal": 15,
    "endsem": 50,
    "total": 100
}

PASSING_PERCENTAGE = 45

# ------------------------------
# Step 1: Load dataset from local system
# ------------------------------

dataset_path = r"C:\Users\hiral\Downloads\student_pass_fail_dataset_10000.csv"
print(f"Using dataset file: {dataset_path}")

# ------------------------------
# Step 2: Load dataset
# ------------------------------

def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path)
        print("Dataset loaded successfully")
        return df
    except Exception as e:
        raise RuntimeError(f"Error loading dataset: {e}")

# ------------------------------
# Step 3: Validate dataset
# ------------------------------

def validate_dataset(df):
    required_columns = []

    for subject in subjects:
        required_columns.extend([
            f"{subject}_midsem",
            f"{subject}_quiz",
            f"{subject}_internal",
            f"{subject}_endsem",
            f"{subject}_total"
        ])

    required_columns.append("attendance_score")

    missing_cols = set(required_columns) - set(df.columns)
    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")

    for subject in subjects:
        if not df[f"{subject}_midsem"].between(0, MARK_LIMITS["midsem"]).all():
            raise ValueError(f"Invalid midsem marks in {subject}")

        if not df[f"{subject}_quiz"].between(0, MARK_LIMITS["quiz"]).all():
            raise ValueError(f"Invalid quiz marks in {subject}")

        if not df[f"{subject}_internal"].between(0, MARK_LIMITS["internal"]).all():
            raise ValueError(f"Invalid internal marks in {subject}")

        if not df[f"{subject}_endsem"].between(0, MARK_LIMITS["endsem"]).all():
            raise ValueError(f"Invalid endsem marks in {subject}")

        if not df[f"{subject}_total"].between(0, MARK_LIMITS["total"]).all():
            raise ValueError(f"Invalid total marks in {subject}")

    if not df["attendance_score"].between(0.0, 10.0).all():
        raise ValueError("Attendance score out of range (0–10)")

    if df.isnull().any().any():
        raise ValueError("Dataset contains missing values")

    print("Dataset validation passed ✔")
    return True

# ------------------------------
# Execute Phase 1
# ------------------------------

df = load_dataset(dataset_path)
validate_dataset(df)

print("PHASE 1 COMPLETED SUCCESSFULLY")

Using dataset file: C:\Users\hiral\Downloads\student_pass_fail_dataset_10000.csv
Dataset loaded successfully
Dataset validation passed ✔
PHASE 1 COMPLETED SUCCESSFULLY


In [26]:
# ==============================
# PHASE 2: SCORE CALCULATION & WEIGHTAGE
# ==============================

def calculate_subject_totals(df):
    """
    Recalculate subject totals to ensure correctness
    """
    for subject in subjects:
        df[f"{subject}_total"] = (
            df[f"{subject}_midsem"]
            + df[f"{subject}_quiz"]
            + df[f"{subject}_internal"]
            + df[f"{subject}_endsem"]
        )
    return df


def calculate_overall_average(df):
    """
    Calculate average marks across all subjects (out of 100)
    """
    total_columns = [f"{subject}_total" for subject in subjects]
    df["overall_marks_avg"] = df[total_columns].mean(axis=1)
    return df


def calculate_final_weighted_score(df):
    """
    Apply 85% marks + 15% attendance weightage
    """
    df["attendance_scaled"] = df["attendance_score"] * 10
    df["final_weighted_score"] = (
        0.85 * df["overall_marks_avg"]
        + 0.15 * df["attendance_scaled"]
    )
    return df


# ------------------------------
# Execute Phase 2
# ------------------------------

df = calculate_subject_totals(df)
df = calculate_overall_average(df)
df = calculate_final_weighted_score(df)

print("PHASE 2 COMPLETED SUCCESSFULLY")

# Preview computed scores
df[[
    "overall_marks_avg",
    "attendance_score",
    "final_weighted_score"
]].head()

PHASE 2 COMPLETED SUCCESSFULLY


,overall_marks_avg,attendance_score,final_weighted_score
0,60.500000,0.13,51.620000
1,57.333333,9.54,63.043333
2,51.166667,3.40,48.591667
3,49.666667,9.32,56.196667
4,58.000000,3.21,54.115000


In [27]:
# ==============================
# PHASE 3: PASS / FAIL CLASSIFICATION
# ==============================

# Safety check
if 'df' not in globals():
    raise NameError("Dataset not available. Run Phase 1 and Phase 2 first.")

PASSING_PERCENTAGE = 45

def classify_pass_fail(df):
    # Subject-wise pass check
    for subject in subjects:
        df[f"{subject}_pass"] = df[f"{subject}_total"] >= PASSING_PERCENTAGE

    # Overall weighted score pass check
    df["overall_pass"] = df["final_weighted_score"] >= PASSING_PERCENTAGE

    # Final result
    df["result"] = (
        df[[f"{subject}_pass" for subject in subjects]]
        .all(axis=1) & df["overall_pass"]
    )

    # Convert boolean to label
    df["result"] = df["result"].map({True: "PASS", False: "FAIL"})

    return df


# Execute Phase 3
df = classify_pass_fail(df)

print("PHASE 3 COMPLETED SUCCESSFULLY")

# Preview results
df[[
    "final_weighted_score",
    "result"
]].head()

PHASE 3 COMPLETED SUCCESSFULLY


,final_weighted_score,result
0,51.620000,PASS
1,63.043333,FAIL
2,48.591667,FAIL
3,56.196667,FAIL
4,54.115000,FAIL


In [28]:
# ==============================
# PHASE 4: RISK ANALYSIS & FAILURE DIAGNOSIS
# ==============================

def calculate_risk_and_reason(row):
    failed_subjects = []
    near_fail_subjects = []

    for subject in subjects:
        score = row[f"{subject}_total"]

        if score < PASSING_PERCENTAGE:
            failed_subjects.append(subject)
        elif PASSING_PERCENTAGE <= score < PASSING_PERCENTAGE + 10:
            near_fail_subjects.append(subject)

    # Base risk from final score
    if row["final_weighted_score"] < PASSING_PERCENTAGE:
        risk = 80
    elif row["final_weighted_score"] < PASSING_PERCENTAGE + 10:
        risk = 60
    elif row["final_weighted_score"] < PASSING_PERCENTAGE + 20:
        risk = 40
    else:
        risk = 20

    # Increase risk for failed or weak subjects
    risk += len(failed_subjects) * 10
    risk += len(near_fail_subjects) * 5

    # Attendance impact
    if row["attendance_score"] < 6:
        risk += 10

    risk = min(risk, 100)

    # Human‑readable reason text
    reason_parts = []
    if failed_subjects:
        reason_parts.append("Failed subjects: " + ", ".join(failed_subjects))
    if near_fail_subjects:
        reason_parts.append("Weak performance in: " + ", ".join(near_fail_subjects))
    if row["attendance_score"] < 6:
        reason_parts.append("Low attendance may have contributed")
    if not reason_parts:
        reason_parts.append("No specific failure reason identified – keep up the good work")

    # Mentor‑like improvement tips
    tips = []
    if failed_subjects:
        tips.append("Seek help from a mentor or tutor on " + ", ".join(failed_subjects))
    if near_fail_subjects:
        tips.append("Strengthen weak areas: " + ", ".join(near_fail_subjects))
    if row["attendance_score"] < 6:
        tips.append("Improve attendance; it influences the weighted score")
    tips.append("Maintain consistency and practice regularly")

    return pd.Series({
        "risk_percent": risk,
        "failed_subjects": failed_subjects if failed_subjects else ["None"],
        "improvement_areas": (

            failed_subjects + near_fail_subjects]].head()

            if (failed_subjects or near_fail_subjects)    "mentor_tips"

            else ["Maintain consistency"]    "reason_text",

        ),    "improvement_areas",

        "reason_text": " | ".join(reason_parts),    "failed_subjects",

        "mentor_tips": " | ".join(tips)    "risk_percent",

    })    "result",

df[[

# Preview

# Apply Phase 4

df[["risk_percent", "failed_subjects", "improvement_areas", "reason_text", "mentor_tips"]] = df.apply(print("PHASE 4 COMPLETED SUCCESSFULLY")

    calculate_risk_and_reason, axis=1
)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 61 (685558388.py, line 63)

In [ ]:
# ==============================
# PHASE 5: FINAL OUTPUT GENERATION
# ==============================

def generate_final_output(row):
    output = {
        "Final Result": row["result"],
        "Risk of Failing Again (%)": row["risk_percent"],
        "Reason of Failure": row.get("reason_text",
                                    row["failed_subjects"] if row["result"] == "FAIL" else "None"),
        "Improvement Suggestions": row.get("mentor_tips", row["improvement_areas"])
    }
    return output


# Apply Phase 5
df["final_output"] = df.apply(generate_final_output, axis=1)

print("PHASE 5 COMPLETED SUCCESSFULLY")

# Show sample final outputs
df["final_output"].head()

PHASE 5 COMPLETED SUCCESSFULLY


0    {'Final Result': 'PASS', 'Risk of Failing Agai...
1    {'Final Result': 'FAIL', 'Risk of Failing Agai...
2    {'Final Result': 'FAIL', 'Risk of Failing Agai...
3    {'Final Result': 'FAIL', 'Risk of Failing Agai...
4    {'Final Result': 'FAIL', 'Risk of Failing Agai...
Name: final_output, dtype: object

In [ ]:
# STEP 6.1: PREPARE DATA FOR ML

# Convert target to numeric
df["result_ml"] = df["result"].map({"PASS": 1, "FAIL": 0})

# Select feature columns
feature_cols = (
    [f"{s}_total" for s in subjects]
    + ["attendance_score", "final_weighted_score"]
)

X = df[feature_cols]
y = df["result_ml"]

print("Feature columns used:")
print(feature_cols)
print("\nTarget distribution:")
print(y.value_counts())

Feature columns used:
['maths_total', 'science_total', 'english_total', 'hindi_total', 'social_science_total', 'computer_science_total', 'attendance_score', 'final_weighted_score']

Target distribution:
result_ml
0    9482
1     518
Name: count, dtype: int64


In [ ]:
# STEP 6.2: TRAIN-TEST SPLIT

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

Training samples: 8000
Testing samples: 2000


In [ ]:
# STEP 6.3: MODEL TRAINING

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print("Model training completed")

Model training completed


In [ ]:
# STEP 6.4: MODEL TESTING & EVALUATION

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", round(accuracy * 100, 2), "%")
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)

Accuracy: 95.5 %

Confusion Matrix:
 [[1880   16]
 [  74   30]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      1896
           1       0.65      0.29      0.40       104

    accuracy                           0.95      2000
   macro avg       0.81      0.64      0.69      2000
weighted avg       0.95      0.95      0.95      2000



In [ ]:
# STEP 7.1: SAVE MODEL

import joblib

joblib.dump(model, "pass_fail_model.pkl")
print("Model saved as pass_fail_model.pkl")

Model saved as pass_fail_model.pkl


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import joblib

model = joblib.load("pass_fail_model.pkl")

subjects = [
    "maths", "science", "english",
    "hindi", "social_science", "computer_science"
]

st.title("🎓 Student Pass/Fail Predictor")

input_data = {}

for subject in subjects:
    input_data[f"{subject}_total"] = st.number_input(
        f"{subject.capitalize()} Total Marks", 0, 100, 50
    )

attendance = st.slider("Attendance Score (0–10)", 0.0, 10.0, 7.0)
input_data["attendance_score"] = attendance

input_data["final_weighted_score"] = (
    0.85 * sum(input_data[f"{s}_total"] for s in subjects) / 6
    + 0.15 * (attendance * 10)
)

if st.button("Predict"):
    df_input = pd.DataFrame([input_data])
    pred = model.predict(df_input)[0]
    prob = model.predict_proba(df_input)[0][1]

    st.subheader("Result")
    st.write("PASS ✅" if pred == 1 else "FAIL ❌")
    st.write("Risk of failing again:", round((1 - prob) * 100, 2), "%")

Writing app.py


In [ ]:
!pip install streamlit pyngrok

   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.1 MB 2.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.1 MB 1.7 MB/s eta 0:00:05
   ----- ---------------------------------- 1.3/9.1 MB 1.9 MB/s eta 0:00:05
   -------- ------------------------------- 1.8/9.1 MB 2.0 MB/s eta 0:00:04
   ----------- ---------------------------- 2.6/9.1 MB 2.2 MB/s eta 0:00:04
   ------------- -------------------------- 3.1/9.1 MB 2.2 MB/s eta 0:00:03
   ---------------- ----------------------- 3.7/9.1 MB 2.2 MB/s eta 0:00:03
   ----------------- ---------------------- 3.9/9.1 MB 2.2 MB/s eta 0:00:03
   -------------------- ------------------- 4.7/9.1 MB 2.3 MB/s eta 0:00:02
   ------------------------ --------------- 5.5/9.1 MB 2.4 MB/s eta 0:00:02
   -------------------------- ---

  You can safely remove it manually.


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(public_url)

t=2026-02-23T12:08:52+0530 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2026-02-23T12:08:52+0530 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2026-02-23T12:08:52+0530 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
!ngrok config add-authtoken 3A1jr34svqU716TPvq0lsL9xVNm_4YDyVovPosezctppeBFK2

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(public_url)

In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

In [ ]:
!streamlit run app.py

In [ ]:
!ls

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd

st.set_page_config(page_title="Pass/Fail Predictor", layout="centered")

st.title("🎓 Student Pass / Fail Predictor")

st.write("This application predicts whether a student will PASS or FAIL based on marks and attendance.")

st.subheader("📂 Dataset Preview")

try:
    df = pd.read_csv("student_pass_fail_dataset_10000.csv")
    st.dataframe(df.head())
except:
    st.warning("Dataset not found. Please upload the dataset.")

st.subheader("✅ Deployment Status")
st.success("Streamlit app is running successfully!")

In [ ]:
!ls

In [ ]:
!streamlit run app.py

^C
